In [2]:
import torch
import numpy as np 
from torch.autograd import Variable
import torch.nn.functional as fun
import matplotlib.pyplot as plt
import torch.utils.data as Data
import torch.nn as nn
import torchvision 
import dgl

In [6]:
#超参数
EPOCH=1
BATCH_SIZE=50
LR=0.001
DOWNLOAD_MINIST=False

In [7]:
#下载数据集
train_data=torchvision.datasets.MNIST(
    root='./mnist',
    train=True,
    transform=torchvision.transforms.ToTensor(), #转为tensor数据(转为灰度图像)
    download=DOWNLOAD_MINIST
)

0it [00:00, ?it/s]

100%|████████████████████████████████████████████████████████████████████▉| 9912320/9912422 [13:51<00:00, 23070.79it/s]

Extracting ./mnist\MNIST\raw\train-images-idx3-ubyte.gz to ./mnist\MNIST\raw



0it [00:00, ?it/s]


 28%|████████████████████▉                                                     | 8192/28881 [00:01<00:01, 16358.06it/s]
32768it [00:01, 21839.53it/s]                                                                                          

0it [00:00, ?it/s]

...

100%|████████████████████████████████████████████████████████████████████▉| 1646592/1648877 [02:35<00:00, 11230.36it/s]

0it [00:00, ?it/s]

Extracting ./mnist\MNIST\raw\t10k-images-idx3-ubyte.gz to ./mnist\MNIST\raw




  0%|                                                                                         | 0/4542 [00:00<?, ?it/s]

Extracting ./mnist\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./mnist\MNIST\raw
Processing...
Done!



1654784it [02:46, 11230.36it/s]                                                                                        

In [8]:
#批训练准备
train_loader=Data.DataLoader(
    dataset=train_data,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=2
)

In [31]:
#得到和提取test data
test_data=torchvision.datasets.MNIST(
    root='./mnist/',
    train=False
)
#注意修改
with torch.no_grad():
    test_x=Variable(torch.unsqueeze(test_data.test_data,dim=1)).type(torch.FloatTensor)[:2000]/255
test_y=test_data.targets[:2000]

In [32]:
#建立神经网络
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.conv1=nn.Sequential(
            nn.Conv2d(         #图片格式(1,28,28)
                in_channels=1, #图片高度
                out_channels=16, #输出高度 (16,28,28)
                kernel_size=5, #felter是5*5的像素点
                stride=1, #每多少步跳一下
                padding=2, #图片外层多加一圈数据
            ),                          #卷积层
            nn.ReLU(),                   #(16,28,28)
            nn.MaxPool2d(kernel_size=2) #池化层 (16,14,14)
        )
        self.conv2=nn.Sequential(   #输入格式(16,14,14)
            nn.Conv2d(16,32,5,1,2),  #卷积后(32,14,14)
            nn.ReLU(),               #(32,14,14)
            nn.MaxPool2d(2)         #池化后(32,7,7)
        )
        self.out=nn.Linear(32*7*7,10)
    def forward(self,x):
        x=self.conv1(x)
        x=self.conv2(x)       #(batch,32,7,7)
        x=x.view(x.size(0),-1)#(batch,32*7*7)  (乘在一起了)
        output=self.out(x)
        return output
         

In [33]:
#初始化
cnn=CNN()
print(cnn)

CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out): Linear(in_features=1568, out_features=10, bias=True)
)


In [34]:
#优化器
optimizer=torch.optim.Adam(cnn.parameters(),lr=LR)
lossfunc=nn.CrossEntropyLoss()

In [41]:
#训练
for epoch in range(EPOCH):
    for step ,(x,y) in enumerate(train_loader):
        b_x=Variable(x)
        b_y=Variable(y)
        
        output=cnn(b_x)
        loss=lossfunc(output,b_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #每50步看一下效果
        if step%50==0:
            test_output=cnn(test_x)
            pred_y=torch.max(test_output,1)[1].data.squeeze()
            accurancy=((pred_y==test_y).sum().item())/test_y.size(0)
            print('Epoch:',epoch,'|train loss:%.4f'%loss.item(),'|test accurancy:',accurancy)

Epoch: 0 |train loss:0.1229 |test accurancy: 0.9665
Epoch: 0 |train loss:0.0528 |test accurancy: 0.9735
Epoch: 0 |train loss:0.0449 |test accurancy: 0.9785
Epoch: 0 |train loss:0.0468 |test accurancy: 0.981
Epoch: 0 |train loss:0.0189 |test accurancy: 0.9775
Epoch: 0 |train loss:0.0237 |test accurancy: 0.976
Epoch: 0 |train loss:0.0301 |test accurancy: 0.977
Epoch: 0 |train loss:0.0092 |test accurancy: 0.9795
Epoch: 0 |train loss:0.0060 |test accurancy: 0.9815
Epoch: 0 |train loss:0.0190 |test accurancy: 0.977
Epoch: 0 |train loss:0.0420 |test accurancy: 0.9755
Epoch: 0 |train loss:0.0795 |test accurancy: 0.979
Epoch: 0 |train loss:0.1104 |test accurancy: 0.9785
Epoch: 0 |train loss:0.0307 |test accurancy: 0.9825
Epoch: 0 |train loss:0.0302 |test accurancy: 0.9805
Epoch: 0 |train loss:0.0693 |test accurancy: 0.98
Epoch: 0 |train loss:0.0387 |test accurancy: 0.981
Epoch: 0 |train loss:0.0328 |test accurancy: 0.9815
Epoch: 0 |train loss:0.0518 |test accurancy: 0.984
Epoch: 0 |train loss:

In [39]:
#打印
test_output=cnn(test_x[:10])
pred_y=torch.max(test_output,1)[1].data.numpy().squeeze()
print(pred_y,'prediction number')
print(test_y[:10].numpy(),'real number')

[7 2 1 0 4 1 4 9 5 9] prediction number
[7 2 1 0 4 1 4 9 5 9] real number
